In [1]:
%run "0_workspace_setup.ipynb" 

In [2]:
from nhm_helpers.output_plots import create_streamflow_plot
from nhm_helpers.nhm_hydrofabric import make_hf_map_elements
from nhm_helpers.map_template import make_streamflow_map
from nhm_helpers.nhm_output_visualization import retrieve_hru_output_info
from ipywidgets import widgets
from IPython.display import display

poi_id_sel = None
crs = 4326

## Introduction
The purpose of this notebook is to assist in the evaluation of NHM subdomain model simulated streamflow. First, the notebook creates a map of gage locations color coded by Kling-Gupta efficiency (KGE; [2009](https://www.sciencedirect.com/science/article/abs/pii/S0022169409004843)) value. Gage locations are overlays in a map of NHM headwater basins (HWs) that are color coded to calibration type: yellow indicates HWs calibrated with statistical streamflow targets at the HW outlet; green indicates HWs that were further calibrated with streamflow observations at selected gage locations.
Next, a gage may be selected and a plot created that shows a time-series of simulated and observed streamflow at daily, monthly, and annual time steps. A flow exceedance curve and table of summary statistics is also provided in the plot for streamflow evaluation purposes. Additionally, maps and plots produced are saved for use outside of notebooks as HTML files in the `html_maps` and `html_plots` folders in the `notebook_output` folder.

The cell below reads the NHM subdomain model hydrofabric elements for mapping purposes using `make_hf_map_elements()`, retrieves `pywatershed` output file information for mapping and plotting using `retrieve_hru_output_info()`, and writes general NHM subdomain model run and hydrofabric information.

In [3]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)
con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

NHM model domain: willamette_river, parameter file: myparam.param
Simulation and observation data range: 01/01/1979 - 12/31/2022 (from control.default.bandit).
 
The parameter file contains 100 gages managed by USGS, 329 segments, and 625 HRUs. 
     564 HRUs are within HWs, and 408 are within HW calibrated with streamflow observations.

## Create an interactive map to evaluate streamflow at gages (pois)
The following cell creates a map of the NHM subdomain model hydrofabric elements and displays monthly Kling-Gupta efficiency (KGE) values for parameter file gages as red (KGE<0.5), yellow (0.5<=KGE<0.7), and green (KGE>=0.7). The map is interactive, made with [folium](https://python-visualization.github.io/folium/v0.18.0/index.html) (see [README](./README.md) for basic map interactive functionality). Use the mouse to left-click on gage icons and display the gage id and gage name. Use the mouse to left-click on HRUs to display the headwater basin (HW) id. HW groupings of HRUs are important when interpreting parameter values and output variables. HRU parameters were grouped by HW and adjusted using a scaling factor during the byHW and byHWobs parts of NHM calibration ([Hay and others, 2023](https://pubs.usgs.gov/tm/06/b10/tm6b10.pdf)). During the byHW calibration part, statistical flow targets at HW outlets were used, and during the subsequent byHWobs part, streamflow obervations were used for selected HWs.

In [4]:
map_file = make_streamflow_map(
    out_dir,
    plot_start_date,
    plot_end_date,
    water_years,
    hru_gdf,
    poi_df,
    poi_id_sel,
    seg_gdf,
    html_maps_dir,
    subdomain,
    HW_basins_gdf,
    HW_basins,
    output_netcdf_filename,
)

KeyError: "not all values found in index 'poi_id'. Try setting the `method` keyword argument (example: method='nearest')."

In [5]:
poi_df

,poi_id,poi_gage_segment,poi_type,nhm_seg,poi_agency,poi_name,latitude,longitude,drainage_area,drainage_area_contrib,nhm_calib,kge
0,14152000,4,1,50136,USGS,"MIDDLE FORK WILLAMETTE RIVER AT JASPER, OR",43.998182,-122.905913,1340.00,NaN,N,0.64
1,14150000,6,1,50138,USGS,"MIDDLE FORK WILLAMETTE RIVER NEAR DEXTER, OR",43.945681,-122.837297,1001.00,NaN,Y,0.58
2,14148000,8,1,50140,USGS,"MF WILLAMETTE RIVER BLW N FORK, NR OAKRIDGE, OR.",43.801235,-122.560888,924.00,NaN,Y,0.66
3,14145500,11,1,50143,USGS,"MF WILLAMETTE RIVER ABV SALT CRK, NEAR OAKRIDG...",43.720957,-122.437826,392.00,NaN,Y,0.61
4,14144800,14,1,50146,USGS,"MIDDLE FORK WILLAMETTE RIVER NR OAKRIDGE, OR",43.601789,-122.457547,264.00,NaN,N,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...
95,14211550,323,0,50457,USGS,"JOHNSON CREEK AT MILWAUKIE, OR",45.452897,-122.643150,53.17,NaN,N,NaN
96,14211500,324,0,50458,USGS,"JOHNSON CREEK AT SYCAMORE, OR",45.477456,-122.508019,26.80,NaN,N,NaN
97,14211400,326,0,50460,USGS,"JOHNSON CREEK AT REGNER ROAD, AT GRESHAM, OR",45.486333,-122.421833,15.36,NaN,N,NaN
98,14211499,327,1,50461,USGS,"KELLEY CREEK AT SE 159TH DRIVE AT PORTLAND, OR",45.476787,-122.498424,4.69,NaN,N,NaN


## Select a gage to evaluate simulated and observed streamflow time-series and streamflow statistics
<font size=4> &#x270D;<font color='green'>**Enter Information:**</font> **Run the cell below. In the resulting drop-down box, select a gage. A plot will be created to evaluate observed and simulated streamflow at the selected gage.** </font><br>
<font size=3> If no gage is selected (default), the first gage listed in the parameter file will be used. The drop-down box selection can be changed and additional plots will be displayed and exported (html files) to<br> `<NHM subdomain model folder>/notebook_output_files/html_plots`.

In [ ]:
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

v = widgets.Combobox(
    # value='John',
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    value=poi_id_sel,
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
)


def on_change(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v.value


v.observe(on_change)
display(v)

<font size=4> &#x1F6D1;If a new selection is made above,</font><br>
<font color='green' size = '3'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

## Make an interactive plot of simulated and observed streamflow and table of streamflow statistics for selected gage
This plot is interactive, and made with [plotly](https://plotly.com/) (see [README](./README.md) for basic plot interactive functionality), and shows subplots of simulated and observed streamflow at daily, monthly, and annual timesteps for the selected gage. Note: the date that appears in popup window when hovering over plotted data reflects the last day of the timestep displayed in the plot. A flow exceedance curve and table of summary statistics is also provided in the plot for streamflow evaluation purposes. 

In [ ]:
plot_file_path = create_streamflow_plot(
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    water_years,
    html_plots_dir,
    output_netcdf_filename,
    out_dir,
    subdomain,
)